In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import ExponentialDecay




In [4]:
# Set the paths for the dataset
dataset_path = '/kaggle/input/yoga-poses-dataset/DATASET/'
train_path = dataset_path + 'TRAIN/'
test_path = dataset_path + 'TEST/'

In [5]:
# Set the image size and batch size
img_size = (224, 224)
batch_size = 32

# Data augmentation for the training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
# Prepare the data generators for train and test sets
train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_path,
                                                  target_size=img_size,
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

Found 1081 images belonging to 5 classes.
Found 470 images belonging to 5 classes.


In [9]:
# Create a CNN model

model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

In [10]:
lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

In [11]:
# Compile the model

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [12]:
# Set callbacks
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

In [13]:
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [14]:
# Train the model
history = model.fit(train_generator,
                    epochs=50,
                    validation_data=test_generator,
                    callbacks=[early_stopping, reduce_lr, model_checkpoint])

Epoch 1/50
 5/34 [===>..........................] - ETA: 27s - loss: 2.5551 - accuracy: 0.2000

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:997: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


34/34 [==============================] - 69s 2s/step - loss: 2.5674 - accuracy: 0.2118 - val_loss: 2.7394 - val_accuracy: 0.1660 - lr: 9.9653e-04
Epoch 2/50
34/34 [==============================] - 45s 1s/step - loss: 2.4449 - accuracy: 0.2118 - val_loss: 2.0686 - val_accuracy: 0.1745 - lr: 9.9297e-04
Epoch 3/50
34/34 [==============================] - 41s 1s/step - loss: 2.2869 - accuracy: 0.1943 - val_loss: 2.1108 - val_accuracy: 0.1851 - lr: 9.8941e-04
Epoch 4/50
34/34 [==============================] - 45s 1s/step - loss: 2.2131 - accuracy: 0.1785 - val_loss: 1.6828 - val_accuracy: 0.2426 - lr: 9.8588e-04
Epoch 5/50
34/34 [==============================] - 44s 1s/step - loss: 2.1099 - accuracy: 0.1998 - val_loss: 1.6019 - val_accuracy: 0.2255 - lr: 9.8235e-04
Epoch 6/50
34/34 [==============================] - 41s 1s/step - loss: 2.0219 - accuracy: 0.2063 - val_loss: 1.7549 - val_accuracy: 0.2170 - lr: 9.7884e-04
Epoch 7/50
34/34 [==============================] - 42s 1s/step - los

In [15]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)


15/15 [==============================] - 13s 921ms/step - loss: 1.5351 - accuracy: 0.3043
Test accuracy: 0.30425530672073364


In [1]:
# Plot the training history
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: name 'plt' is not defined

In [2]:

def predict_yoga_pose(image_path):
    # Load the image
    image = load_img(image_path, target_size=img_size)

    # Convert the image to an array
    image_array = img_to_array(image)
    image_array = image_array / 255.0  # Rescale the image

    # Expand the dimensions of the image array
    image_array = np.expand_dims(image_array, axis=0)

    # Make predictions
    predictions = model.predict(image_array)

    # Decode the predictions to get the class label
    predicted_class = np.argmax(predictions, axis=-1)
    class_labels = list(train_generator.class_indices.keys())
    predicted_label = class_labels[predicted_class[0]]

    return predicted_label